In [101]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import copy 
import pickle
import requests

import urllib3

import numpy as np 
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data_dir = '/Users/song/Downloads/merian'

brick_file = 'legacysurvey-1933m005-ccds.fits'

ccd_cat = Table.read(os.path.join(data_dir, brick_file))

print(len(ccd_cat))

64


In [129]:
class Brick(object):
    '''A base class containing codes to handle data for a DECaLS brick.
    
    '''
    LEGACY_DATA = "https://portal.nersc.gov/project/cosmo/data/legacysurvey"
    LEGACY_VIEW = "https://www.legacysurvey.org/viewer"
    
    def __init__(self, brick_id, dr=8):
        '''
        Creat a Brick object based on the brick ID.
        
        Brick ID has the format <AAAa>c<BBB> where A, a and B are digits and c is either the letter m or p (e.g. 1126p222). 
        The names are derived from the (RA, Dec) center of the brick. The first four digits are int(RA×10), 
        followed by p to denote positive Dec or m to denote negative Dec ("plus"/"minus"), followed by three digits of int(Dec×10). 
        For example the case 1126p222 corresponds to (RA, Dec) = (112.6°, +22.2°).
        
        Image stacks are on tangent-plane (WCS TAN) projections, 3600 × 3600 pixels, at 0.262 arcseconds per pixel.
        '''
        self.id = brick_id
        
        # Directory that stores all the coadd data
        self.coadd_url = self.LEGACY_DATA + "/dr{:d}/south/coadd/{:s}/{:s}/".format(
            int(dr), self.id[0:3], self.id)
        
        # API to get the CCD data
        self.ccd_img_api = self.LEGACY_VIEW + '/image-data/ls-dr{:d}/'.format(int(dr))
        self.ccd_var_api = self.LEGACY_VIEW + '/iv-data/ls-dr{:d}/'.format(int(dr))
        self.ccd_msk_api = self.LEGACY_VIEW + '/dq-data/ls-dr{:d}/'.format(int(dr))
        
        # URL of the CCD catalog
        self.ccd_cat_url = self.coadd_url + "legacysurvey-{:s}-ccds.fits".format(self.id)
        self.ccd_cat = Table(fits.open(self.ccd_cat_url)[1].data)
        self.ccd_num = len(self.ccd_cat) 
    
    def _download_fits(self, url, output, overwrite=True):
        '''Download a FITS file from the LegacySurvey directory.'''
        # Check whether the file exists
        r = requests.head(url)
        if r.status_code == requests.codes.ok:
            _ = fits.open(url).writeto(output, overwrite=overwrite)
            return True
        else:
            print("# Can not download file: {:s}".format(url))
            return False
    
    def _download_jpg(self, url, output):
        '''Download a JPG figure.'''
        r = requests.head(url)
        if r.status_code == requests.codes.ok:
            img_data = requests.get(url).content
            with open(output, 'wb') as handler:
                handler.write(img_data)
            return True
        else:
            print("# Can not download file: {:s}".format(url))
            return False
        
    def download_coadd(self, file_type, loc, fz=True, suffix=None):
        '''Download a coadd data product for the brick.'''
        if suffix is None:
            suffix = 'fits.fz' if fz else 'fits'
        file_name = "legacysurvey-{:s}-{:s}.{:s}".format(self.id, file_type, suffix)
        output = os.path.join(loc, file_name)
        
        if 'fits' in suffix:
            _ = self._download_fits(self.coadd_url + file_name, output)
        elif 'jpg' in suffix:
            _ = self._download_jpg(self.coadd_url + file_name, output)
        else:
            raise Exception("Wrong suffix. Either fits/fits.fz or jpg.")
        
    def download_ccd(self, ccd, loc):
        '''Download the reduced image, variance, and mask for one CCD.'''
        ccd_str = "{:s}-{:d}-{:s}-{:s}".format(
            ccd['camera'].strip(), ccd['expnum'], ccd['ccdname'].strip(), ccd['filter'])
        
        _ = self._download_fits(self.ccd_img_api + ccd_str, os.path.join(loc, ccd_str + '_img.fits'))
        _ = self._download_fits(self.ccd_var_api + ccd_str, os.path.join(loc, ccd_str + '_var.fits'))
        _ = self._download_fits(self.ccd_msk_api + ccd_str, os.path.join(loc, ccd_str + '_msk.fits'))
    
    def get_decam_coadd(self, loc, model=False):
        '''Get all the basic DECam coadd images for the brick.'''
        # Get the image, invarance, and model images
        if model:
            file_types = ['image', 'invvar', 'model']
        else:
            file_types = ['image', 'invvar']
        filters = ['g', 'r', 'z']
        
        if not os.path.exists(loc):
            os.makedirs(loc, exist_ok=True)
        
        for filt_type in ['{:s}-{:s}'.format(t, f) for t in file_types for f in filters]:
            _ = self.download_coadd(filt_type, loc, fz=True)
        
        # Download the maskbits file
        _ = self.download_coadd('maskbits', loc, fz=True)
        
        # Download the JPG figure
        _ = self.download_coadd('image', loc, suffix='jpg')
        _ = self.download_coadd('model', loc, suffix='jpg')
    
    def get_decam_ccds(self, loc, band=None):
        '''Download the CCD data.'''
        if not os.path.exists(loc):
            os.makedirs(loc, exist_ok=True)
            
        if band is not None and band.strip() in ['g', 'r', 'z']:
            ccd_use = self.ccd_cat[self.ccd_cat['filter'] == band.strip()]
            if len(ccd_use) > 0:
                for ccd in ccd_use:
                    _ = self.download_ccd(ccd, loc)
        else:
            for ccd in self.ccd_cat:
                _ = self.download_ccd(ccd, loc)
        
    def save_ccd_cat(self, loc):
        '''Save the CCD catalog to a FITS file.'''
        cat_name = "legacysurvey-{:s}-ccds.fits".format(self.id)
        self.ccd_cat.write(os.path.join(loc, cat_name), overwrite=True)
    
        
    

In [130]:
brick_id = '1933m005'

data_dir = '/Users/song/Downloads/merian'

brick_loc = os.path.join(data_dir, brick_id)

In [131]:
brick_test = Brick(brick_id)

In [118]:
brick_test.get_decam_coadd(os.path.join(brick_loc, 'coadd'), model=True)

In [122]:
brick_test.save_ccd_cat(brick_loc)

In [132]:
brick_test.get_decam_ccds(os.path.join(brick_loc, 'ccd', 'g'), band='g')

In [133]:
brick_test.get_decam_ccds(os.path.join(brick_loc, 'ccd', 'r'), band='r')

In [134]:
brick_test.get_decam_ccds(os.path.join(brick_loc, 'ccd', 'z'), band='z')